# Load Dataset Using ImageFolder

In [3]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader # `DataLoader` to load data in batches and enable shuffling

In [4]:
transform=transforms.Compose([
    transforms.Resize((150,150)),  # Resize all images to 150x150 pixels
    transforms.ToTensor()          # Convert PIL Image or numpy.ndarray to a PyTorch tensor
])



In [5]:
# Load training dataset from the "chest_xray/train" directory and apply the transformations
train_data=datasets.ImageFolder("chest_xray/train",transform=transform)

In [6]:

val_data=datasets.ImageFolder("chest_xray/val",transform=transform)

In [7]:
# Load test dataset from the "chest_xray/test" directory and apply the transformations
test_data = datasets.ImageFolder("chest_xray/test", transform=transform)

In [8]:
# Create a DataLoader for the training data
# - batch_size=32: load 32 images at a time
# - shuffle=True: shuffle the data at every epoch to improve generalization
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

In [9]:
# Create a DataLoader for the validation data
# - shuffle=False: do not shuffle, as it's not needed for evaluation
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

In [10]:
# Create a DataLoader for the test data
# - shuffle=False: keep the order of test data consistent for evaluation
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# Build the CNN Model

In [11]:
# Import necessary modules from PyTorch
import torch.nn as nn                 # Contains building blocks like layers
import torch.nn.functional as F      # Contains functions like ReLU, sigmoid, etc.

In [12]:
# Define a CNN model for pneumonia detection
class PneumoniaCNN(nn.Module):
    def __init__(self):
        super(PneumoniaCNN,self).__init__()# Initialize the base class nn.Module
        # First convolutional layer:
        # - Input channels: 3 (RGB image)
        # - Output channels: 32
        # - Kernel size: 3x3
        self.conv1 = nn.Conv2d(3, 32, 3)

        # Max pooling layer with:
        # - Kernel size: 2x2
        # - Stride: 2
        self.pool = nn.MaxPool2d(2, 2)

        # Second convolutional layer:
        # - Input channels: 32
        # - Output channels: 64
        # - Kernel size: 3x3
        self.conv2 = nn.Conv2d(32, 64, 3)
        self._to_linear = 64 * 36 * 36

        # Fully connected layer:
        # - Input features: 64 * 35 * 35 (flattened feature map size after conv and pooling)
        # - Output features: 128
        self.fc1 = nn.Linear(self._to_linear, 128)

        # Dropout layer with 50% probability to prevent overfitting
        self.dropout = nn.Dropout(0.5)

        # Output layer:
        # - Input features: 128
        # - Output features: 1 (binary classification: pneumonia or not)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        # Apply first convolution, ReLU activation, and max pooling
        # Output shape after conv1: (32, 148, 148) -> after pool: (32, 74, 74)
        x = self.pool(F.relu(self.conv1(x)))

         # Apply second convolution, ReLU activation, and max pooling
        # Output shape after conv2: (64, 72, 72) -> after pool: (64, 35, 35)
        x = self.pool(F.relu(self.conv2(x)))

         # Flatten the tensor to feed into the fully connected layer
        x = x.view(x.size(0), -1)

         # Apply ReLU activation to the first fully connected layer
        x = F.relu(self.fc1(x))

         # Apply dropout to reduce overfitting
        x = self.dropout(x)

        # Output layer with sigmoid to get a probability (binary classification)
        x = torch.sigmoid(self.fc2(x))

        # Return the final prediction
        return x




# Train the Mode

In [13]:
# Import torch and the optimizer module
import torch
from torch import optim


In [14]:
# Set the device to GPU if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



In [15]:
# Initialize the model and move it to the selected device (GPU/CPU)
model = PneumoniaCNN().to(device)

In [16]:
# Define the loss function:
# - BCELoss is used for binary classification problems (outputs between 0 and 1)
criterion = nn.BCELoss()

In [17]:
# Define the optimizer:
# - Adam optimizer with a learning rate of 0.001
# - It will update the model's parameters during training
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [18]:
# Training loop for 5 epochs
for epoch in range(5):
    model.train()  # Set the model to training mode (enables dropout, batchnorm, etc.)
    running_loss = 0.0  # To accumulate loss for the epoch
 
    # Iterate over batches of images and labels from the training DataLoader
    for images, labels in train_loader:
        # Move images and labels to the selected device
        images = images.to(device)
        # Convert labels to float and reshape to (batch_size, 1) to match model output
        labels = labels.float().to(device).unsqueeze(1)

        # Zero out gradients from the previous step
        optimizer.zero_grad()

        # Perform a forward pass through the model
        outputs = model(images)

        # Compute the binary cross-entropy loss between predictions and actual labels
        loss = criterion(outputs, labels)

        # Backpropagate the loss to compute gradients
        loss.backward()

        # Update the model's weights using the optimizer
        optimizer.step()

        # Add the current batch's loss to the running total
        running_loss += loss.item()

    # Print average loss for the epoch
    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader):.4f}")


Epoch 1, Loss: 0.4288
Epoch 2, Loss: 0.2549
Epoch 3, Loss: 0.2337
Epoch 4, Loss: 0.1959
Epoch 5, Loss: 0.1616


# Evaluate the Model

In [19]:
# Import the classification_report function to generate a text summary of precision, recall, F1-score, etc.
from sklearn.metrics import classification_report

# Set the model to evaluation mode (important for models with dropout or batch norm)
model.eval()

# Initialize empty lists to store true labels and predicted labels
y_true, y_pred = [], []

# Disable gradient calculation to save memory and computations during inference
with torch.no_grad():
    # Iterate over the test dataset in batches
    for images, labels in test_loader:
        # Move the images to the same device as the model (CPU or GPU)
        images = images.to(device)

        # Get model outputs (logits or probabilities), move to CPU, and convert to NumPy array
        outputs = model(images).cpu().numpy()

        # Apply threshold to outputs to get binary predictions (1 if > 0.5 else 0), and flatten to 1D
        preds = (outputs > 0.5).astype(int).flatten()

        # Append predictions to the predicted labels list
        y_pred.extend(preds)

        # Convert true labels to NumPy array and append to the true labels list
        y_true.extend(labels.numpy())


# Print a detailed classification report comparing true vs. predicted labels
# Includes precision, recall, F1-score for each class (Normal and Pneumonia)
print(classification_report(y_true, y_pred, target_names=['Normal', 'Pneumonia']))

              precision    recall  f1-score   support

      Normal       0.95      0.36      0.52       234
   Pneumonia       0.72      0.99      0.83       390

    accuracy                           0.75       624
   macro avg       0.84      0.67      0.68       624
weighted avg       0.81      0.75      0.72       624



In [20]:
torch.save(model.state_dict(), "pneumonia_cnn.pt")
